In [10]:
class Qipan:
    def __init__(self,x,y):
        self.x = x
        self.y = y
        self.block_list = []
        for i in range(x):
            hang = []
            for j in range(y):
                hang.append(Block(i,j))
            self.block_list.append(hang)
            
    #在此定义函数,对block的改变为block_list[i][j].f = ?
    def set_start(self,i,j,has_start):
        if has_start:
            for xx in range(self.x):
                for yy in range(self.y):
                    if self.block_list[xx][yy].sta == 1:
                        self.block_list[xx][yy].sta = 0
                        self.block_list[xx][yy].update(self.block_list[xx][yy].f,self.block_list[xx][yy].g,self.block_list[xx][yy].h)
                        #print("起点更换成功")
        else:
            #print("起点设置成功")
            pass
        #注意这里是[j][i],第j行第i个距左侧i个格子上侧j个格子
        self.block_list[j][i].sta = 1
        self.block_list[j][i].update(self.block_list[j][i].f,self.block_list[j][i].g,self.block_list[j][i].h)
        
    def set_end(self,i,j,has_end):
        if has_end:
            for xx in range(self.x):
                for yy in range(self.y):
                    if self.block_list[xx][yy].sta == 2:
                        self.block_list[xx][yy].sta = 0
                        self.block_list[xx][yy].update(self.block_list[xx][yy].f,self.block_list[xx][yy].g,self.block_list[xx][yy].h)
                        #print("终点更换成功")
        else:
            #print("终点更换成功")
            pass
        self.block_list[j][i].sta = 2
        self.block_list[j][i].update(self.block_list[j][i].f,self.block_list[j][i].g,self.block_list[j][i].h)
            
    def set_ob(self,i,j):
        if self.block_list[j][i].sta == 1:
            self.block_list[j][i].sta = 3
            self.block_list[j][i].update(self.block_list[j][i].f,self.block_list[j][i].g,self.block_list[j][i].h)
            return 1
        if self.block_list[j][i].sta == 2:
            self.block_list[j][i].sta = 3
            self.block_list[j][i].update(self.block_list[j][i].f,self.block_list[j][i].g,self.block_list[j][i].h)
            return 2
        else:
            self.block_list[j][i].sta = 3
            self.block_list[j][i].update(self.block_list[j][i].f,self.block_list[j][i].g,self.block_list[j][i].h)
            print(j,i,"设置障碍成功")
            return 0
        
    def update_all(self):
        for xx in range(self.x):
            for yy in range(self.y):
                self.block_list[xx][yy].update(self.block_list[xx][yy].f,self.block_list[xx][yy].g,self.block_list[xx][yy].h)
        
        
        
class Block:
    def __init__(self,x,y,f = 0,g = 0,h = 0):
        self.x = x
        self.y = y
        self.f = f
        self.h = h
        self.g = g
        self.sta = 0 #0为无状态，1为起点，2为终点，3为障碍，，，#4为最佳路径,5为正在搜索即在open list中，6为已搜索过在close list中
        self.display_fgh = False
        self.color = self.get_color()
        #父方格号，（-1，-1）代表暂时还没有
        self.parent = (-1,-1)
        
    def update(self,f,g,h):
        self.color = self.get_color()
        self.f = f
        self.h = h
        self.g = g
        
    def get_color(self):
        if self.sta == 0:
            return [255,255,255]
        if self.sta == 1:
            return [0,255,0]
        if self.sta == 2:
            return [255,0,0]
        if self.sta == 3:
            return [0,0,0]
        if self.sta == 4:
            return [255,255,0]
        if self.sta == 5:
            return [135,206,250]
        if self.sta == 6:
            return [0,191,255]

In [17]:
#display函数，状态0为确认前，不显示g、h、f值；状态1为确认后,显示g、h、f值
def display(qipan,screen,status = 0):
    #先定义字体
    fgh_font=pg.font.Font(None,20)
    #白屏
    screen.fill((255,255,255))
    for i in range(qipan.x):
        for j in range(qipan.y):
            rect_list=[i*BLOCK_LEN,j*BLOCK_LEN,BLOCK_LEN,BLOCK_LEN]
            #print(rect_list)
            #先画出格子
            pygame.draw.rect(screen,[0,0,0],rect_list,2)
            block_list = [i*BLOCK_LEN+2,j*BLOCK_LEN+2,BLOCK_LEN-4,BLOCK_LEN-4]
            #填充格子内颜色
            pygame.draw.rect(screen,qipan.block_list[i][j].color,block_list,0)
              
    if status ==1:
        for i in range(qipan.x):
            for j in range(qipan.y):
                #写上f,g,h
                if qipan.block_list[i][j].display_fgh:
                    font_g = str(qipan.block_list[i][j].g)
                    font_surf_g = fgh_font.render(font_g,True,(0,0,0)) 
                    sur_rect_g = font_surf_g.get_rect()
                    #定义中心位置
                    sur_rect_g.center = (i*BLOCK_LEN+15,BLOCK_LEN*j+10)
                    #把字画屏幕上
                    screen.blit(font_surf_g,sur_rect_g)
                    font_f = str(qipan.block_list[i][j].f)
                    font_surf_f = fgh_font.render(font_f,True,(0,0,0)) 
                    sur_rect_f = font_surf_f.get_rect()
                    sur_rect_f.center = (i*BLOCK_LEN+15,BLOCK_LEN*j+BLOCK_LEN-10)
                    screen.blit(font_surf_f,sur_rect_f)
                    font_h = str(qipan.block_list[i][j].h)
                    font_surf_h = fgh_font.render(font_h,True,(0,0,0)) 
                    sur_rect_h = font_surf_h.get_rect()
                    sur_rect_h.center = (i*BLOCK_LEN+BLOCK_LEN-15,BLOCK_LEN*j+BLOCK_LEN-10)
                    screen.blit(font_surf_h,sur_rect_h)
                start_image=pygame.image.load(r'D:\360data\重要数据\桌面\static\start.png')
                place1 = [qipan.x*BLOCK_LEN/4-50,qipan.y*BLOCK_LEN+10]
                screen.blit(start_image,place1)
                end_image=pygame.image.load(r'D:\360data\重要数据\桌面\static\restart.png')
                place2 = [qipan.x*BLOCK_LEN/4*3-50,qipan.y*BLOCK_LEN+10]
                screen.blit(end_image,place2)
                ob_image=pygame.image.load(r'D:\360data\重要数据\桌面\static\step.png')
                place3 = [qipan.x*BLOCK_LEN/4-50,qipan.y*BLOCK_LEN+60]
                screen.blit(ob_image,place3)
                con_image=pygame.image.load(r'D:\360data\重要数据\桌面\static\result.png')
                place4 = [qipan.x*BLOCK_LEN/4*3-50,qipan.y*BLOCK_LEN+60]
                screen.blit(con_image,place4)
                start_image=pygame.image.load(r'D:\360data\重要数据\桌面\static\statement2.png')
                place_s = [qipan.x*BLOCK_LEN/2-190,qipan.y*BLOCK_LEN+110]
                screen.blit(start_image,place_s)
                pygame.display.flip()
                
    if status == 0:
        start_image=pygame.image.load(r'D:\360data\重要数据\桌面\static\starting_point.png')
        place1 = [qipan.x*BLOCK_LEN/4-50,qipan.y*BLOCK_LEN+10]
        screen.blit(start_image,place1)
        end_image=pygame.image.load(r'D:\360data\重要数据\桌面\static\ending_point.png')
        place2 = [qipan.x*BLOCK_LEN/4*3-50,qipan.y*BLOCK_LEN+10]
        screen.blit(end_image,place2)
        ob_image=pygame.image.load(r'D:\360data\重要数据\桌面\static\obstacle.png')
        place3 = [qipan.x*BLOCK_LEN/4-50,qipan.y*BLOCK_LEN+60]
        screen.blit(ob_image,place3)
        con_image=pygame.image.load(r'D:\360data\重要数据\桌面\static\confirm.png')
        place4 = [qipan.x*BLOCK_LEN/4*3-50,qipan.y*BLOCK_LEN+60]
        screen.blit(con_image,place4)
        start_image=pygame.image.load(r'D:\360data\重要数据\桌面\static\statement.png')
        place_s = [qipan.x*BLOCK_LEN/2-190,qipan.y*BLOCK_LEN+110]
        screen.blit(start_image,place_s)
        pygame.display.flip()

In [18]:
def is_empty_ol(qipan):
    flag = True
    for i in range(qipan.x):
        for j in range(qipan.y):
            if qipan.block_list[i][j].sta ==5:
                flag = False
    return flag

#sta = 1时返回起点坐标，2返回终点坐标
def get_point(qipan,sta):
    for i in range(qipan.x):
        for j in range(qipan.y):
            if qipan.block_list[i][j].sta ==sta:
                return i,j
            
def ol_find_min(qipan):
    min_value = 99999
    x = -1
    y = -1
    for i in range(qipan.x):
        for j in range(qipan.y):
            if qipan.block_list[i][j].sta == 5 and qipan.block_list[i][j].g<min_value:
                min_value = qipan.block_list[i][j].g
                x = i
                y = j
    return x,y

def judge_end(qipan,min_i,min_j):
    end_i,end_j = get_point(qipan,2)
    if min_i-1<=end_i<=min_i+1 and min_j-1<=end_j<=min_j+1:
        qipan.block_list[end_i][end_j].parent = (min_i,min_j)
        print("找到最佳路径")
        return True
    elif is_empty_ol(qipan):
        print("失败，起点被障碍包围了")
        return True
    else:
        return False
    
def find_best_route(qipan):
    end_i,end_j = get_point(qipan,2)
    start_i,start_j = get_point(qipan,1)
    x,y = qipan.block_list[end_i][end_j].parent[0],qipan.block_list[end_i][end_j].parent[1]
    while x != start_i or y != start_j:
        qipan.block_list[x][y].sta = 4
        x,y = qipan.block_list[x][y].parent[0],qipan.block_list[x][y].parent[1]

In [19]:
def step_search(qipan):
    start_i,start_j = get_point(qipan,1)
    end_i,end_j = get_point(qipan,2)
    #先判断open_list是否为空
    if is_empty_ol(qipan):
        #把起点周围的方格(除障碍)放入open_list，起点放入close_list
        qipan.block_list[start_i][start_j].h = 10*(abs(start_i-end_i)+abs(start_j-end_j))
        qipan.block_list[start_i][start_j].g = qipan.block_list[start_i][start_j].f+qipan.block_list[start_i][start_j].h
        #左上
        if qipan.block_list[start_i-1][start_j-1].sta != 3 and start_i-1>-1and start_j-1>-1:    #后面两个and是防止起点在边缘时超过下限-1
            qipan.block_list[start_i-1][start_j-1].display_fgh = True
            qipan.block_list[start_i-1][start_j-1].sta =5
            qipan.block_list[start_i-1][start_j-1].f =14
            qipan.block_list[start_i-1][start_j-1].h = 10*(abs(start_i-1-end_i)+abs(start_j-1-end_j))
            qipan.block_list[start_i-1][start_j-1].g = qipan.block_list[start_i-1][start_j-1].f+qipan.block_list[start_i-1][start_j-1].h
            qipan.block_list[start_i-1][start_j-1].parent = (start_i,start_j)
        #左
        if qipan.block_list[start_i-1][start_j].sta != 3 and start_i-1>-1:
            qipan.block_list[start_i-1][start_j].display_fgh = True
            qipan.block_list[start_i-1][start_j].sta =5
            qipan.block_list[start_i-1][start_j].f =10
            qipan.block_list[start_i-1][start_j].h = 10*(abs(start_i-1-end_i)+abs(start_j-end_j))
            qipan.block_list[start_i-1][start_j].g = qipan.block_list[start_i-1][start_j].f+qipan.block_list[start_i-1][start_j].h
            qipan.block_list[start_i-1][start_j].parent = (start_i,start_j)
        #左下
        if start_j+1<qipan.y:
            if qipan.block_list[start_i-1][start_j+1].sta != 3 and start_i-1>-1:
                qipan.block_list[start_i-1][start_j+1].display_fgh = True
                qipan.block_list[start_i-1][start_j+1].sta =5
                qipan.block_list[start_i-1][start_j+1].f =14
                qipan.block_list[start_i-1][start_j+1].h = 10*(abs(start_i-1-end_i)+abs(start_j+1-end_j))
                qipan.block_list[start_i-1][start_j+1].g = qipan.block_list[start_i-1][start_j+1].f+qipan.block_list[start_i-1][start_j+1].h
                qipan.block_list[start_i-1][start_j+1].parent = (start_i,start_j)
        #上
        if qipan.block_list[start_i][start_j-1].sta != 3 and start_j-1>-1:
            qipan.block_list[start_i][start_j-1].display_fgh = True
            qipan.block_list[start_i][start_j-1].sta =5
            qipan.block_list[start_i][start_j-1].f =10
            qipan.block_list[start_i][start_j-1].h = 10*(abs(start_i-end_i)+abs(start_j-1-end_j))
            qipan.block_list[start_i][start_j-1].g = qipan.block_list[start_i][start_j-1].f+qipan.block_list[start_i][start_j-1].h
            qipan.block_list[start_i][start_j-1].parent = (start_i,start_j)
        #下
        if start_j+1<qipan.y:
            if qipan.block_list[start_i][start_j+1].sta != 3:
                qipan.block_list[start_i][start_j+1].display_fgh = True
                qipan.block_list[start_i][start_j+1].sta =5
                qipan.block_list[start_i][start_j+1].f =10
                qipan.block_list[start_i][start_j+1].h = 10*(abs(start_i-end_i)+abs(start_j+1-end_j))
                qipan.block_list[start_i][start_j+1].g = qipan.block_list[start_i][start_j+1].f+qipan.block_list[start_i][start_j+1].h
                qipan.block_list[start_i][start_j+1].parent = (start_i,start_j)
        #右上
        if start_i+1<qipan.x:
            if qipan.block_list[start_i+1][start_j-1].sta != 3 and start_j-1>-1:
                qipan.block_list[start_i+1][start_j-1].display_fgh = True
                qipan.block_list[start_i+1][start_j-1].sta =5
                qipan.block_list[start_i+1][start_j-1].f =14
                qipan.block_list[start_i+1][start_j-1].h = 10*(abs(start_i+1-end_i)+abs(start_j-1-end_j))
                qipan.block_list[start_i+1][start_j-1].g = qipan.block_list[start_i+1][start_j-1].f+qipan.block_list[start_i+1][start_j-1].h
                qipan.block_list[start_i+1][start_j-1].parent = (start_i,start_j)
        #右
        if start_i+1<qipan.x:
            if qipan.block_list[start_i+1][start_j].sta != 3:
                qipan.block_list[start_i+1][start_j].display_fgh = True
                qipan.block_list[start_i+1][start_j].sta =5
                qipan.block_list[start_i+1][start_j].f =10
                qipan.block_list[start_i+1][start_j].h = 10*(abs(start_i+1-end_i)+abs(start_j-end_j))
                qipan.block_list[start_i+1][start_j].g = qipan.block_list[start_i+1][start_j].f+qipan.block_list[start_i+1][start_j].h
                qipan.block_list[start_i+1][start_j].parent = (start_i,start_j)
        #右下
        if start_i+1<qipan.x and start_j+1<qipan.y:
            if qipan.block_list[start_i+1][start_j+1].sta != 3:
                qipan.block_list[start_i+1][start_j+1].display_fgh = True
                qipan.block_list[start_i+1][start_j+1].sta =5
                qipan.block_list[start_i+1][start_j+1].f =14
                qipan.block_list[start_i+1][start_j+1].h = 10*(abs(start_i+1-end_i)+abs(start_j+1-end_j))
                qipan.block_list[start_i+1][start_j+1].g = qipan.block_list[start_i+1][start_j+1].f+qipan.block_list[start_i+1][start_j+1].h
                qipan.block_list[start_i+1][start_j+1].parent = (start_i,start_j)
        return False
    else:
        #找open_list中f最小的
        min_i,min_j = ol_find_min(qipan)
        qipan.block_list[min_i][min_j].sta = 6
        if qipan.block_list[min_i-1][min_j-1].sta not in[1,2,3,6] and min_i-1>-1and min_j-1>-1:
            #如果该点本来不在open_list中，让它加入open_list,给它f,g,h值
            if qipan.block_list[min_i-1][min_j-1].sta != 5:
                qipan.block_list[min_i-1][min_j-1].display_fgh = True
                qipan.block_list[min_i-1][min_j-1].sta = 5
                qipan.block_list[min_i-1][min_j-1].f = qipan.block_list[min_i][min_j].f + 14
                qipan.block_list[min_i-1][min_j-1].h = 10*(abs(min_i-1-end_i)+abs(min_j-1-end_j))
                qipan.block_list[min_i-1][min_j-1].g = qipan.block_list[min_i-1][min_j-1].f+qipan.block_list[min_i-1][min_j-1].h
                qipan.block_list[min_i-1][min_j-1].parent = (min_i,min_j)
            #本来就在open_list中，检查f,g,h值是否要更新
            else:
                new_f = qipan.block_list[min_i][min_j].f + 14
                if new_f<qipan.block_list[min_i-1][min_j-1].f:
                    qipan.block_list[min_i-1][min_j-1].f = new_f
                    qipan.block_list[min_i-1][min_j-1].g = qipan.block_list[min_i-1][min_j-1].f+qipan.block_list[min_i-1][min_j-1].h
                    qipan.block_list[min_i-1][min_j-1].parent = (min_i,min_j)
        if qipan.block_list[min_i-1][min_j].sta not in[1,2,3,6] and min_i-1>-1:
            if qipan.block_list[min_i-1][min_j].sta != 5:
                qipan.block_list[min_i-1][min_j].display_fgh = True
                qipan.block_list[min_i-1][min_j].sta = 5
                qipan.block_list[min_i-1][min_j].f = qipan.block_list[min_i][min_j].f + 10
                qipan.block_list[min_i-1][min_j].h = 10*(abs(min_i-1-end_i)+abs(min_j-end_j))
                qipan.block_list[min_i-1][min_j].g = qipan.block_list[min_i-1][min_j].f+qipan.block_list[min_i-1][min_j].h
                qipan.block_list[min_i-1][min_j].parent = (min_i,min_j)
            else:
                new_f = qipan.block_list[min_i][min_j].f + 10
                if new_f<qipan.block_list[min_i-1][min_j].f:
                    qipan.block_list[min_i-1][min_j].f = new_f
                    qipan.block_list[min_i-1][min_j].g = qipan.block_list[min_i-1][min_j-1].f+qipan.block_list[min_i-1][min_j-1].h
                    qipan.block_list[min_i-1][min_j].parent = (min_i,min_j)
        if min_j+1<qipan.y:
            if qipan.block_list[min_i-1][min_j+1].sta not in[1,2,3,6] and min_i-1>-1:
                if qipan.block_list[min_i-1][min_j+1].sta != 5:
                    qipan.block_list[min_i-1][min_j+1].display_fgh = True
                    qipan.block_list[min_i-1][min_j+1].sta = 5
                    qipan.block_list[min_i-1][min_j+1].f = qipan.block_list[min_i][min_j].f + 14
                    qipan.block_list[min_i-1][min_j+1].h = 10*(abs(min_i-1-end_i)+abs(min_j+1-end_j))
                    qipan.block_list[min_i-1][min_j+1].g = qipan.block_list[min_i-1][min_j+1].f+qipan.block_list[min_i-1][min_j+1].h
                    qipan.block_list[min_i-1][min_j+1].parent = (min_i,min_j)
                #本来就在open_list中，检查f,g,h值是否要更新
                else:
                    new_f = qipan.block_list[min_i][min_j].f + 14
                    if new_f<qipan.block_list[min_i-1][min_j+1].f:
                        qipan.block_list[min_i-1][min_j+1].f = new_f
                        qipan.block_list[min_i-1][min_j+1].g = qipan.block_list[min_i-1][min_j+1].f+qipan.block_list[min_i-1][min_j+1].h
                        qipan.block_list[min_i-1][min_j+1].parent = (min_i,min_j)
        if qipan.block_list[min_i][min_j-1].sta not in[1,2,3,6] and min_j-1>-1:
            if qipan.block_list[min_i][min_j-1].sta != 5:
                qipan.block_list[min_i][min_j-1].display_fgh = True
                qipan.block_list[min_i][min_j-1].sta = 5
                qipan.block_list[min_i][min_j-1].f = qipan.block_list[min_i][min_j].f + 10
                qipan.block_list[min_i][min_j-1].h = 10*(abs(min_i-end_i)+abs(min_j-1-end_j))
                qipan.block_list[min_i][min_j-1].g = qipan.block_list[min_i][min_j-1].f+qipan.block_list[min_i][min_j-1].h
                qipan.block_list[min_i][min_j-1].parent = (min_i,min_j)
            else:
                new_f = qipan.block_list[min_i][min_j].f + 10
                if new_f<qipan.block_list[min_i][min_j-1].f:
                    qipan.block_list[min_i][min_j-1].f = new_f
                    qipan.block_list[min_i][min_j-1].g = qipan.block_list[min_i][min_j-1].f+qipan.block_list[min_i][min_j-1].h
                    qipan.block_list[min_i][min_j-1].parent = (min_i,min_j)
        if min_j+1<qipan.y:
            if qipan.block_list[min_i][min_j+1].sta not in[1,2,3,6]:
                if qipan.block_list[min_i][min_j+1].sta != 5:
                    qipan.block_list[min_i][min_j+1].display_fgh = True
                    qipan.block_list[min_i][min_j+1].sta = 5
                    qipan.block_list[min_i][min_j+1].f = qipan.block_list[min_i][min_j].f + 10
                    qipan.block_list[min_i][min_j+1].h = 10*(abs(min_i-end_i)+abs(min_j+1-end_j))
                    qipan.block_list[min_i][min_j+1].g = qipan.block_list[min_i][min_j+1].f+qipan.block_list[min_i][min_j+1].h
                    qipan.block_list[min_i][min_j+1].parent = (min_i,min_j)
                else:
                    new_f = qipan.block_list[min_i][min_j].f + 10
                    if new_f<qipan.block_list[min_i][min_j+1].f:
                        qipan.block_list[min_i][min_j+1].f = new_f
                        qipan.block_list[min_i][min_j+1].g = qipan.block_list[min_i][min_j+1].f+qipan.block_list[min_i][min_j+1].h
                        qipan.block_list[min_i][min_j+1].parent = (min_i,min_j)
        if min_i+1<qipan.x:
            if qipan.block_list[min_i+1][min_j-1].sta not in[1,2,3,6]and min_j-1>-1:
                if qipan.block_list[min_i+1][min_j-1].sta != 5:
                    qipan.block_list[min_i+1][min_j-1].display_fgh = True
                    qipan.block_list[min_i+1][min_j-1].sta = 5
                    qipan.block_list[min_i+1][min_j-1].f = qipan.block_list[min_i][min_j].f + 14
                    qipan.block_list[min_i+1][min_j-1].h = 10*(abs(min_i+1-end_i)+abs(min_j-1-end_j))
                    qipan.block_list[min_i+1][min_j-1].g = qipan.block_list[min_i+1][min_j-1].f+qipan.block_list[min_i+1][min_j-1].h
                    qipan.block_list[min_i+1][min_j-1].parent = (min_i,min_j)
                #本来就在open_list中，检查f,g,h值是否要更新
                else:
                    new_f = qipan.block_list[min_i][min_j].f + 14
                    if new_f<qipan.block_list[min_i+1][min_j-1].f:
                        qipan.block_list[min_i+1][min_j-1].f = new_f
                        qipan.block_list[min_i+1][min_j-1].g = qipan.block_list[min_i+1][min_j-1].f+qipan.block_list[min_i+1][min_j-1].h
                        qipan.block_list[min_i+1][min_j-1].parent = (min_i,min_j)
        if min_i+1<qipan.x:
            if qipan.block_list[min_i+1][min_j].sta not in[1,2,3,6]:
                if qipan.block_list[min_i+1][min_j].sta != 5:
                    qipan.block_list[min_i+1][min_j].display_fgh = True
                    qipan.block_list[min_i+1][min_j].sta = 5
                    qipan.block_list[min_i+1][min_j].f = qipan.block_list[min_i][min_j].f + 10
                    qipan.block_list[min_i+1][min_j].h = 10*(abs(min_i+1-end_i)+abs(min_j-end_j))
                    qipan.block_list[min_i+1][min_j].g = qipan.block_list[min_i+1][min_j].f+qipan.block_list[min_i+1][min_j].h
                    qipan.block_list[min_i+1][min_j].parent = (min_i,min_j)
                else:
                    new_f = qipan.block_list[min_i][min_j].f + 10
                    if new_f<qipan.block_list[min_i+1][min_j].f:
                        qipan.block_list[min_i+1][min_j].f = new_f
                        qipan.block_list[min_i+1][min_j].g = qipan.block_list[min_i+1][min_j].f+qipan.block_list[min_i+1][min_j].h
                        qipan.block_list[min_i+1][min_j].parent = (min_i,min_j)
        if min_i+1<qipan.x and min_j+1<qipan.y:       
            if qipan.block_list[min_i+1][min_j+1].sta  not in[1,2,3,6]:
                if qipan.block_list[min_i+1][min_j+1].sta != 5:
                    qipan.block_list[min_i+1][min_j+1].display_fgh = True
                    qipan.block_list[min_i+1][min_j+1].sta = 5
                    qipan.block_list[min_i+1][min_j+1].f = qipan.block_list[min_i][min_j].f + 14
                    qipan.block_list[min_i+1][min_j+1].h = 10*(abs(min_i+1-end_i)+abs(min_j+1-end_j))
                    qipan.block_list[min_i+1][min_j+1].g = qipan.block_list[min_i+1][min_j+1].f+qipan.block_list[min_i+1][min_j+1].h
                    qipan.block_list[min_i+1][min_j+1].parent = (min_i,min_j)
                else:
                    new_f = qipan.block_list[min_i][min_j].f + 14
                    if new_f<qipan.block_list[min_i+1][min_j+1].f:
                        qipan.block_list[min_i+1][min_j+1].f = new_f
                        qipan.block_list[min_i+1][min_j+1].g = qipan.block_list[min_i+1][min_j+1].f+qipan.block_list[min_i+1][min_j+1].h
                        qipan.block_list[min_i+1][min_j+1].parent = (min_i,min_j)
        flag = judge_end(qipan,min_i,min_j)
        #返回标志，是否已结束
        return flag

In [20]:
import pygame
import sys
import pygame as pg
import time
def main(t = 0.3):
    flag = 1
    while(flag):
        x=int(input("请输入迷宫的长："))
        y=int(input("请输入迷宫的宽："))
        if(x<4 or y<4):
            print("迷宫的长和宽必须大于三，请重新输入：")
            continue
        flag = 0
    pg.init()
    pg.font.init()
    screen=pg.display.set_mode((BLOCK_LEN*x,BLOCK_LEN*y+BLANK_HEIGHT))
    screen.fill((255,255,255))
    pg.display.set_caption("迷宫")
    qipan = Qipan(x,y)
    display(qipan,screen)
    print("事件循环")
    step = 0 #step为1：设置起点；2：设置终点；3：设置障碍
    has_start = False #还未设置起点
    has_end = False #还未设置终点
    period = 1
    end_flag = False #二阶段结束标志
    while True:
        if period ==1:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    #print("退出")
                    pygame.quit()
                    sys.exit()
                if event.type==pygame.MOUSEBUTTONDOWN: #判断鼠标位置以及是否摁了下去。
                    #做需要做的事情，如开始游戏。
                    x, y = pygame.mouse.get_pos()
                    j = int(x/BLOCK_LEN)
                    i = int(y/BLOCK_LEN)
                    #点的是第i行第j个
                    if qipan.x*BLOCK_LEN/4-50<=x<=qipan.x*BLOCK_LEN/4+50 and qipan.y*BLOCK_LEN+10<=y<=qipan.y*BLOCK_LEN+50:
                        step = 1
                        #print("设置起点")
                    if qipan.x*BLOCK_LEN/4*3-50<=x<=qipan.x*BLOCK_LEN/4*3+50 and qipan.y*BLOCK_LEN+10<=y<=qipan.y*BLOCK_LEN+50:
                        step = 2
                        #print("设置终点")
                    if qipan.x*BLOCK_LEN/4-50<=x<=qipan.x*BLOCK_LEN/4+50 and qipan.y*BLOCK_LEN+60<=y<=qipan.y*BLOCK_LEN+100:
                        step = 3
                        #print("设置障碍")
                    if qipan.x*BLOCK_LEN/4*3-50<=x<=qipan.x*BLOCK_LEN/4*3+50 and qipan.y*BLOCK_LEN+60<=y<=qipan.y*BLOCK_LEN+100:
                        if has_start and has_end:
                            #print("确认成功，进入阶段二")
                            period = 2
                            first_time = True
                        else:
                            #print("确认失败")
                            pass
                    if 0<=x<=qipan.x*BLOCK_LEN and 0<=y<=qipan.y*BLOCK_LEN:
                        print("点击了迷宫")
                        if step == 0:
                            #print("无事件")
                            pass
                        elif step == 1:
                            qipan.set_start(i,j,has_start)
                            has_start = True
                        elif step == 2:
                            qipan.set_end(i,j,has_end)
                            has_end = True
                        elif step == 3:
                            special = qipan.set_ob(i,j)
                            if special == 1:
                                has_start = False
                                #print("将起点换成障碍")
                            if special == 2:
                                has_end = False
                                #print("将终点换成障碍")
                    display(qipan,screen)
                else:
                    #print("阶段一，无事件")
                    time.sleep(t)
        if period == 2:
            if first_time:
                qipan.update_all()
                display(qipan,screen,1)
                first_time =False
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    #print("退出")
                    pygame.quit()
                    sys.exit()
                elif event.type==pygame.MOUSEBUTTONDOWN: #判断鼠标位置以及是否摁了下去。
                    #做需要做的事情，如开始游戏。
                    x, y = pygame.mouse.get_pos()
                    j = int(x/BLOCK_LEN)
                    i = int(y/BLOCK_LEN)
                    #开始
                    if qipan.x*BLOCK_LEN/4-50<=x<=qipan.x*BLOCK_LEN/4+50 and qipan.y*BLOCK_LEN+10<=y<=qipan.y*BLOCK_LEN+50:
                        for i in range(qipan.x):
                            for j in range(qipan.y):
                                if qipan.block_list[i][j].sta == 1:
                                    qipan.block_list[i][j].display_fgh = True
                    #重新开始
                    elif qipan.x*BLOCK_LEN/4*3-50<=x<=qipan.x*BLOCK_LEN/4*3+50 and qipan.y*BLOCK_LEN+10<=y<=qipan.y*BLOCK_LEN+50:
                        #重设标志
                        end_flag = False
                        for i in range(qipan.x):
                            for j in range(qipan.y):
                                if qipan.block_list[i][j].sta in [4,5,6]:
                                    qipan.block_list[i][j].sta = 0
                                qipan.block_list[i][j].display_fgh = False
                                #清零
                                qipan.block_list[i][j].f = 0
                                qipan.block_list[i][j].g = 0
                                qipan.block_list[i][j].h = 0
                                qipan.block_list[i][j].parent = (-1,-1)
                        for i in range(qipan.x):
                            for j in range(qipan.y):
                                if qipan.block_list[i][j].sta == 1:
                                    qipan.block_list[i][j].display_fgh = True
                    elif qipan.x*BLOCK_LEN/4-50<=x<=qipan.x*BLOCK_LEN/4+50 and qipan.y*BLOCK_LEN+60<=y<=qipan.y*BLOCK_LEN+100:
                        #print(end_flag)
                        if not end_flag:
                            end_flag = step_search(qipan)
                        else:  #找到最佳路径或无解
                            end_i,end_j = get_point(qipan,2)
                            if qipan.block_list[end_i][end_j].parent ==(-1,-1):
                                pass
                            else:
                                find_best_route(qipan)
                    elif qipan.x*BLOCK_LEN/4*3-50<=x<=qipan.x*BLOCK_LEN/4*3+50 and qipan.y*BLOCK_LEN+60<=y<=qipan.y*BLOCK_LEN+100:
                        while not end_flag:
                            end_flag = step_search(qipan)
                        end_i,end_j = get_point(qipan,2)
                        if qipan.block_list[end_i][end_j].parent ==(-1,-1):
                            pass
                        else:
                            find_best_route(qipan)
                    qipan.update_all()
                    display(qipan,screen,1)
                else:
                    #print("阶段二，无事件")
                    time.sleep(t)

In [21]:
TIME = 0    #设置得越小游戏卡顿越少
BLOCK_LEN = 80
BLANK_HEIGHT = 170


In [23]:
if __name__ =="__main__":
    main(TIME)

请输入迷宫的长：20
请输入迷宫的宽：6
事件循环
点击了迷宫
点击了迷宫
点击了迷宫
9 0 设置障碍成功
点击了迷宫
9 1 设置障碍成功
点击了迷宫
9 2 设置障碍成功
点击了迷宫
9 3 设置障碍成功
点击了迷宫
9 4 设置障碍成功
点击了迷宫
8 1 设置障碍成功
点击了迷宫
7 1 设置障碍成功
点击了迷宫
6 1 设置障碍成功
点击了迷宫
3 3 设置障碍成功
点击了迷宫
3 2 设置障碍成功
点击了迷宫
3 4 设置障碍成功
点击了迷宫
15 1 设置障碍成功
点击了迷宫
15 2 设置障碍成功
点击了迷宫
15 3 设置障碍成功
找到最佳路径
找到最佳路径


SystemExit: 

d:\Users\PC\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
